*This notebook is part of  course materials for CS 345: Machine Learning Foundations and Practice at Colorado State University.
Original versions were created by Asa Ben-Hur.
The content is availabe [on GitHub](https://github.com/asabenhur/CS345).*

*The text is released under the [CC BY-SA license](https://creativecommons.org/licenses/by-sa/4.0/), and code is released under the [MIT license](https://opensource.org/licenses/MIT).*

<img style="padding: 10px; float:right;" alt="CC-BY-SA icon.svg in public domain" src="https://upload.wikimedia.org/wikipedia/commons/d/d0/CC-BY-SA_icon.svg" width="125">


<a href="https://colab.research.google.com/github//asabenhur/CS345/blob/master/notebooks/module05_03_model_selection.ipynb">
  <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%matplotlib inline
%autosave 0
import numpy as np
import matplotlib.pyplot as plt

Autosave disabled


# Model selection using nested cross-validation

This notebook covers the topic of selecting classifier parameters using nested cross-validation cross validation.

As discussed in an earlier notebook, whenever a classifier has hyperparameters that require setting, the appropriate protocol for evaluating classifier accuracy is as follows:

* Divide the data into separate train, validation, and test sets
* Loop over all combinations of values of hyperparameters
* For each value combination, train a model on the training set and evaluate it on the validation set
* Choose best performing set of parameters and retrain the classifier using the training and validation set.
* Evaluate the resulting classifier on the test set

Now that we know how to perform cross-validation, we can update this protocol to make use of this technique for more efficient use of our data.

### Model selection using grid search in scikit-learn

Before we describe the procedure for model selection using cross-validation, we will introduce scikit-learn functionality to perform model selection using grid-search, which will simplify our code for performing this task.

First, let's load some data:

In [35]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X = data.data
y = data.target

In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.neighbors import KNeighborsClassifier

First, let's do plain cross-validation, without trying to choose a particular value for the parameters:

In [33]:
classifier = KNeighborsClassifier()

cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

accuracy = cross_val_score(classifier, X, y, cv=cv, scoring='accuracy')
np.mean(accuracy)

0.9580952380952381

Now let's see if we can do better if we choose better parameters than the defaults:

In [5]:
from sklearn.model_selection import GridSearchCV

# for the nearest neighbor classifier we can select multiple 
# hyperparameters:
# n_neighbors - number of neighbors to base classification on
# p - whether to use the L1 or L2 norm when computing distances
# weights - the sklearn KNN implementation provides the option to
# weight the contribution of each example uniformly or inversely 
# proportional to its distance
# this leads to the following grid of values:

param_grid = {'n_neighbors': [3,5,7,9,11],
              'p': [1, 2], 
              'weights': ['uniform', 'distance']}

classifier = GridSearchCV(KNeighborsClassifier(), param_grid)

classifier.fit(X, y);

When you `fit` a `GridSearchCV` runs the following grid-search procedure:

**Training using grid-search**:

**Input**:  a classifier and a set of parameters with a set of values to try for each parameter.

* Loop over all combinations of classifier parameter.
* For each value of the classifier parameters, perform cross-validation on the training data and evaluate accuracy
* Use the best scoring model parameters, and retrain on the entire dataset 

The trained `GridSearchCV` object contains information regarding the best parameter values and detailed cross-validation results:

In [6]:
classifier.best_params_

{'n_neighbors': 9, 'p': 1, 'weights': 'uniform'}

In [7]:
classifier.cv_results_

{'mean_fit_time': array([0.00124049, 0.00113034, 0.00106826, 0.00092564, 0.00074859,
        0.00070038, 0.00070176, 0.00084171, 0.00071735, 0.00067835,
        0.00135517, 0.00067902, 0.00067387, 0.00070939, 0.0012753 ,
        0.00132437, 0.00096903, 0.00076356, 0.00071392, 0.00068421]),
 'std_fit_time': array([2.46249023e-04, 4.91560785e-04, 3.62914731e-04, 1.62758262e-04,
        4.95331224e-05, 4.64099019e-05, 3.22338392e-05, 3.44222090e-04,
        5.94701011e-05, 8.03919732e-06, 1.28981066e-03, 1.26949778e-05,
        1.57555541e-05, 3.23949704e-05, 4.98421735e-04, 3.42567142e-04,
        2.10586892e-04, 1.83854310e-04, 4.13763337e-05, 2.29300672e-05]),
 'mean_score_time': array([0.00868835, 0.00336967, 0.01099401, 0.0034924 , 0.00599141,
        0.00165248, 0.00529981, 0.00162029, 0.00501184, 0.00165544,
        0.00502276, 0.00159011, 0.00520353, 0.00176578, 0.01083288,
        0.00357895, 0.00838418, 0.00176077, 0.00498481, 0.00163655]),
 'std_score_time': array([1.75679409e-

Note that the ``fit`` method of the ``GridSearchCV`` classifier performed cross-validation to find the best parameters.  This is sometimes called external cross-validation. We can now easily performed **nested cross-validation** simply by performing cross-validation over the ``GridSearchCV`` classifier:

In [34]:
accuracy = cross_val_score(classifier, X, y, cv=cv, scoring='accuracy')
np.mean(accuracy)

0.9580952380952381


Our original procedure which uses train/validation/test set splits is shown here:

* Divide the data into separate train, validation, and test sets
* Loop over all combinations of values of hyperparameters
* For each value combination, train a model on the training set and evaluate it on the validation set
* Choose best performing set of parameters and retrain the classifier using the training and validation set.
* Evaluate the resulting classifier on the test set

It is replaced replaced with the following version which uses cross-validation instead:

* Divide the data into $k$ subsets
* Loop over the $k$ subset, and at each iteration one of them serves as a test set.
  * Loop over all combinations of values of the hyperparameters
  * For each value combination, evaluate the classifier using cross-validation
  * Choose best performing set of parameters and re-train the classifier using all the training data for that fold.
  * Evaluate the resulting classifier on the test set for the given fold
* Combine accuracy estimates over all the folds.


<img style="padding: 10px; float:center;" alt="nested cross validation; created by Asa Ben-Hur inspired by figure from https://github.com/rasbt/python-machine-learning-book-3rd-edition/blob/master/ch06/ch06.ipynb" src="https://github.com/asabenhur/CS345/raw/master/notebooks/figures/nested_cross_validation.jpg" width="600">

This process looks quite expensive.  Let's be more concrete and evaluate how many models we need to train when considering say 100 combinations of hyperparameter values and performing 5-fold cross-validation in both the inner and outer loops.
For fitting the grid-search object: for each combination of hyperparameter values we run 5-fold cross-validation, resulting in training 500 models.
If we are doing nested cross validation the overall number of models is $100 \times 5 \times 5 = 2,500$.
The process of cross-validation and nested cross-validation is highly parallelizable since folds can be trained and evaluated independently of each other for each combination of values of the hyperparameters.


### Exercise

* Perform model selection for decision trees to select the optimal value of the `max_depth` and `min_samples_split` hyperparameters using scikit-learn's [decision tree implementation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). The `min_samples_split` is the minimum number of samples required to split an internal node in the tree. For each parameter choose a wide enough range of values that makes sense.

**Note**:
If a full grid search is too expensive, you can choose to perform randomized grid search, which randomly chooses points from the grid of values.  This is implemented in scikit-learn as the class [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV).  In neural networks, which have a very large number of parameters a complete grid search is not feasible, and randomized grid search is the only viable choice.

### Obtaining unbiased results by preventing information leak between train and test data

When selecting one or two hyperparameters, the bias in accuracy when comparing correct experimental protocol with a protocol that allows for information leakage between the train and test sets can be quite small.
However, in situations where a large number of hyper-parameters needs to be selected the difference can be dramatic.  Consider as an example the problem of feature selection.  In this problem we can think of each feature as being associated with a hyper-parameter that controls whether that feature should be included.

We will demonstrate this using a dataset describing the activity of a large number of genes, from which disease state can be inferred.  The particular dataset we will analyze looks at biological samples taken from leukemia patients with two types of leukemia: acute myeloid leukemia (AML) and acute lymphoblastic leukemia (ALL).  The data was taken from the following publication:

* Golub, Todd R., et al. "Molecular classification of cancer: class discovery and class prediction by gene expression monitoring." Science  (1999): 531-537.

In [27]:
import requests
link = "https://web.stanford.edu/~hastie/CASI_files/DATA/leukemia_big.csv"
# retrieve the contents of the file
contents = requests.get(link)
lines = contents.text.split()
# the data is in csv format and the labels appear in the first 
# row of the dataset
class_convert = {'ALL':1, 'AML':0}
y = np.array([class_convert[token] for token in lines[0].split(',')])
X = np.array([ [float(token) for token in line.split(',')] 
              for line in lines[1:] ])
X = X.transpose()
X.shape,y.shape

((72, 7128), (72,))

First let's evaluate the accuracy of the scikit-learn perceptron using all the features:

In [20]:
from sklearn.linear_model import Perceptron
p = Perceptron()
accuracy = cross_val_score(p, X, y, cv=cv, scoring='accuracy')
np.mean(accuracy)

0.9038095238095238

Next, let's perform feature selection on the entire dataset (even though it's the wrong way to do):

In [26]:
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline

# feature selection using RFE
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html
selector = RFE(Perceptron(), step=0.1, n_features_to_select=20)

# perform feature selection on the data
X_sel = selector.fit_transform(X, y)
print ('number of features: ', X_sel.shape[1])

accuracy = cross_val_score(p, X_sel, y, cv=cv, scoring='accuracy')
np.mean(accuracy)

number of features:  20


0.9457142857142857

We performed feature selection using a method called **Recursive Feature Elimination** (RFE).  RFE builds on the intuition that the magnitude of the weight vector of a linear classifier is a good indication of a feature's usefulness.
Rather than simply removing the features with the lowest value of the weight vector, RFE alternates between training a linear classifier and removing a small number of features with the lowest weight vector magnitude.  RFE was proposed in the following publication:

> Guyon, I., Weston, J., Barnhill, S., & Vapnik, V., “Gene selection for cancer classification using support vector machines”, Mach. Learn., 46(1-3), 389–422, 2002.

As proposed, it uses support vector machines as the base classifier, but it can be used with any linear classifier.


Scikit-learn provides us with functionality for easily performing feature selection the correct way, which leads to reduced accuracy:

In [24]:
selector = RFE(Perceptron(), step=0.1, n_features_to_select=20)

pipeline = Pipeline(
    [('feature_selector', selector),
     ('classifier', Perceptron())])
accuracy = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')
np.mean(accuracy)


0.9190476190476191

The choice of using 20 features was arbitrary and not optimal as we can see by performing grid search over the number of selected features:

In [28]:
grid_search = GridSearchCV(pipeline, 
                          {'feature_selector__n_features_to_select' : 
                           [5, 10, 20, 30, 40, 50]})
accuracy = cross_val_score(grid_search, X, y, cv=cv, scoring='accuracy')
np.mean(accuracy)

0.9457142857142857